In [33]:
#!pip install flask_ngrok
import pandas as pd
import pandas as pd
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify
import flask
from sklearn.ensemble import RandomForestRegressor

In [24]:
df = pd.read_csv("DIT_DATA_AGGRS.csv")
df["year"] = df.ts.apply(lambda x: x[:4])
df["month"] = df.ts.apply(lambda x: int(x[5:7]))
df["day"] = df.ts.apply(lambda x: int(x[8:10]))
df = df.drop("ts", axis=1)
df.to_csv("DIT_DATA_AGGRS.csv", index=False)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def home():

    try:
      df = pd.read_csv("DIT_DATA_AGGRS.csv")
      df = df[df["zid"]==int(flask.request.args.get("zid"))].sample(50)

      regr = RandomForestRegressor(max_depth=2)
      regr.fit(df.drop("customers_cnt_total_sum", axis=1), df["customers_cnt_total_sum"])
      customers_cnt_total_sum = regr.predict(df.sample(1).drop("customers_cnt_total_sum", axis=1).to_numpy())[0]

      regr = RandomForestRegressor(max_depth=2)
      regr.fit(df.drop("customers_cnt_long_sum", axis=1), df["customers_cnt_long_sum"])
      customers_cnt_long_sum = regr.predict(df.sample(1).drop("customers_cnt_long_sum", axis=1).to_numpy())[0]

      regr = RandomForestRegressor(max_depth=2)
      regr.fit(df.drop("customers_cnt_work_sum", axis=1), df["customers_cnt_work_sum"])
      customers_cnt_work_sum = regr.predict(df.sample(1).drop("customers_cnt_work_sum", axis=1).to_numpy())[0]

      regr = RandomForestRegressor(max_depth=2)
      regr.fit(df.drop("customers_cnt_loc_sum", axis=1), df["customers_cnt_loc_sum"])
      customers_cnt_loc_sum = regr.predict(df.sample(1).drop("customers_cnt_loc_sum", axis=1).to_numpy())[0]

      return jsonify({"customers_cnt_loc_sum": round(customers_cnt_loc_sum), 
                    "customers_cnt_work_sum": round(customers_cnt_work_sum), 
                    "customers_cnt_long_sum": round(customers_cnt_long_sum),
                    "customers_cnt_total_sum": round(customers_cnt_total_sum)
                    })
    except:
      return jsonify({"customers_cnt_loc_sum": "", 
                    "customers_cnt_work_sum": "", 
                    "customers_cnt_long_sum": "",
                    "customers_cnt_total_sum": ""
                    })

app.run()